# About this notebook  
- PyTorch resnext50_32x4d -> tf_efficientnet_b0
- No TTA
- baseline notebook is [here](https://www.kaggle.com/yasufuminakama/cassava-resnext50-32x4d-starter-inference)


## Train Info.
- 256 Image Size
- 3 folds -> 4 folds
- Train notebook : [[Train] SETI-BL: Pytorch Starter🔥](https://www.kaggle.com/piantic/train-seti-bl-pytorch-starter)


## Try more
- Use different image size
- Ensemble other difference models
- TTA
- etc.


If this notebook is helpful, feel free to upvote :)

And please upvote the original notebook as well.

`V3` - Initial version

`V4` - Use 3 folds for inference

`V5` - Add `CFG.mode` for `channel` and `spatial`. Use effb0 and 4 folds for inference
> you can get lb 0.96+.

`V7` - `Train notebook` was updated, so change some parts for `4` mode types.
> `spatial_6ch`, `spatial_3ch`, `6_channel`, `3_channel`

`V9` - Fix some bugs

# Directory settings

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # specify GPUs locally

OUTPUT_DIR = './'
MODEL_DIR = '../input/seti-resnext50-32x4d-weights/sample7/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# CFG

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    num_workers=8
    encoder='tf_efficientnet_b0' #[resnext50_32x4d, tf_efficientnet_b0 ]
    size=256
    batch_size=128
    seed=2020
    target_size=1
    target_col='target'
    n_fold=4
    trn_fold=[0,1,2,3]
    inference=True
    mode='spatial_6ch' #['spatial_6ch', 'spatial_3ch', '6_channel', '3_channel']

# Library

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = roc_auc_score(y_true, y_pred)
    return score


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'inference.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

#LOGGER = init_logger()

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

# Data Loading

In [ ]:
test = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
test['file_path'] = test['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')
display(test.head())

# Dataset

In [ ]:
# ====================================================
# Dataset
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df[CFG.target_col].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = np.load(self.file_names[idx])
        if CFG.mode == 'spatial_6ch':
            image = image.astype(np.float32)
            image = np.vstack(image).transpose((1, 0))
        elif CFG.mode == 'spatial_3ch':
            image = image[::2].astype(np.float32)
            image = np.vstack(image).transpose((1, 0))
        elif CFG.mode == '6_channel':
            image = image.astype(np.float32)
            image = np.transpose(image, (1,2,0))
        elif CFG.mode == '3_channel':
            image = image[::2].astype(np.float32)
            image = np.transpose(image, (1,2,0))
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()

        return image

# Transforms

In [ ]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size*3),
            ToTensorV2(),
        ])

# MODEL

In [ ]:
# ====================================================
# MODEL
# ====================================================
class SETImodel(nn.Module):
    def __init__(self, model_name=CFG.encoder, pretrained=False):
        super().__init__()
        if CFG.mode == 'spatial_6ch' or CFG.mode == 'spatial_3ch':
            self.encoder = timm.create_model(model_name, pretrained=pretrained, in_chans=1)
        else:
            self.encoder = timm.create_model(model_name, pretrained=pretrained, in_chans=6)

        if hasattr(self.encoder, "fc"):
            nb_ft = self.encoder.fc.in_features
            self.encoder.fc = nn.Linear(nb_ft, CFG.target_size)
        elif hasattr(self.encoder, "_fc"):
            nb_ft = self.encoder._fc.in_features
            self.encoder._fc = nn.Linear(nb_ft, CFG.target_size)
        elif hasattr(self.encoder, "classifier"):
            nb_ft = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Linear(nb_ft, CFG.target_size)
        elif hasattr(self.encoder, "last_linear"):
            nb_ft = self.encoder.last_linear.in_features
            self.encoder.last_linear = nn.Linear(nb_ft, CFG.target_size)

    def forward(self, x):
        x = self.encoder(x)
        return x

# Helper functions

In [ ]:
# ====================================================
# Helper functions
# ====================================================
def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state)
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
            y_preds = y_preds.sigmoid().to('cpu').numpy()
            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

# inference

In [ ]:
# ====================================================
# inference
# ====================================================
model = SETImodel(model_name=CFG.encoder, pretrained=False)
states = [torch.load(MODEL_DIR+f'{CFG.encoder}_fold{fold}_best_score.pth')['state_dict'] for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)
predictions = inference(model, states, test_loader, device)

In [ ]:
# submission
test['target'] = predictions
test[['id', 'target']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
test.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.hist(test.target,bins=100);